In [ ]:
pip install chromadb

In [2]:
import numpy as np
# np.object = object 
# np.bool = bool
# np.int = int

from angle_emb import AnglE, Prompts
angle = AnglE.from_pretrained('BAAI/bge-large-en-v1.5').cuda()
# angle.set_prompt(prompt=Prompts.C)

def getEmbeddingsFunction(txt):
#     return angle.encode({'text': txt}, to_numpy=True)
    return  angle.encode(txt)[0].tolist()

def get_embeddings(texts):
    embeddings = []
    for text in texts:
        # Replace this with your actual getEmbeddings function
        embedding = getEmbeddingsFunction(text)
        embeddings.append(embedding)
    return list(embeddings)


In [3]:
%%time
import gensim
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Define your sourceTexts and queryTexts arrays
sourceTexts = [
    "US tops 5 million confirmed virus cases",
    "Canada's last fully intact ice shelf has suddenly collapsed, forming a Manhattan-sized iceberg",
    "Beijing mobilises invasion craft along coast as Taiwan tensions escalate",
    "The National Park Service warns against sacrificing slower friends in a bear attack",
    "Maine man wins $1M from $25 lottery ticket",
    "Make huge profits without work, earn up to $100,000 a day",
    "i want to fuck you",
    "Check your account balence",
    "you look haat"
]

queryTexts =  [
   "im so horney",
    "im feeling lucky",
    "i like nature and animals",
    "i feel cold",
    "im scared",
    "stocks"
]

# Get embeddings for sourceTexts and queryTexts
source_embeddings = get_embeddings(sourceTexts)
query_embeddings = get_embeddings(queryTexts)


CPU times: user 574 ms, sys: 35.4 ms, total: 609 ms
Wall time: 604 ms


In [4]:
import chromadb
from chromadb.config import Settings
chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))

In [5]:
try:
    chroma_client.delete_collection(name="my_collection")
except:
    print('Collection not found')

Collection not found


In [6]:
collection = chroma_client.create_collection(name="my_collection")

In [7]:
type(source_embeddings[0])
type(source_embeddings[0][0])

float

In [8]:
collection.add(
    embeddings=source_embeddings,
    documents=sourceTexts,
#     metadatas=[{"source": "one"}, {"source": "two"}],
    ids= ["{:d}".format(x) for x in range(0,len(sourceTexts))]
)

In [13]:
collection.query(
    query_embeddings=[query_embeddings[0],query_embeddings[2]],
    n_results=2,
#     where={"source": "one"}
)

{'ids': [['8', '6'], ['8', '3']],
 'distances': [[246.87991333007812, 268.5283203125],
  [268.9467468261719, 288.8773498535156]],
 'metadatas': [[None, None], [None, None]],
 'embeddings': None,
 'documents': [['you look haat', 'i want to fuck you'],
  ['you look haat',
   'The National Park Service warns against sacrificing slower friends in a bear attack']],
 'uris': None,
 'data': None}

In [6]:
# Perform semantic search for each query
for i, query_text in enumerate(queryTexts):
    print(f"QueryText: {query_text}")
    query_vector = query_embeddings[i]

    # Calculate cosine similarity scores between the query and source texts
    similarity_scores = [cosine_similarity(query_vector, source_vector)[0][0] for source_vector in source_embeddings]

    # Sort the source texts by similarity score and get the top 3 matches
    sorted_matches = sorted(enumerate(similarity_scores), key=lambda x: x[1], reverse=True)[:3]

    # Print the top 3 matches
    for rank, (index, score) in enumerate(sorted_matches, start=1):
        print(f"Match{rank}: {sourceTexts[index]} (Similarity Score: {score:.4f})")
    print()

QueryText: im so horney
Match1: i want to fuck you (Similarity Score: 0.4509)
Match2: you look haat (Similarity Score: 0.4087)
Match3: Check your account balence (Similarity Score: 0.4033)

QueryText: im feeling lucky
Match1: Maine man wins $1M from $25 lottery ticket (Similarity Score: 0.4928)
Match2: Make huge profits without work, earn up to $100,000 a day (Similarity Score: 0.4625)
Match3: i want to fuck you (Similarity Score: 0.4489)

QueryText: i like nature and animals
Match1: The National Park Service warns against sacrificing slower friends in a bear attack (Similarity Score: 0.5142)
Match2: i want to fuck you (Similarity Score: 0.4978)
Match3: Canada's last fully intact ice shelf has suddenly collapsed, forming a Manhattan-sized iceberg (Similarity Score: 0.4017)

QueryText: i feel cold
Match1: Canada's last fully intact ice shelf has suddenly collapsed, forming a Manhattan-sized iceberg (Similarity Score: 0.4358)
Match2: Check your account balence (Similarity Score: 0.4188)


In [2]:
%%time
from angle_emb import AnglE, Prompts

angle = AnglE.from_pretrained('BAAI/bge-large-en-v1.5').cuda()
# angle = AnglE.from_pretrained('sentence-transformers/all-MiniLM-L6-v2').cuda()
angle.set_prompt(prompt=Prompts.C)


import gensim
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Define your sourceTexts and queryTexts arrays
sourceTexts = [
    "US tops 5 million confirmed virus cases",
    "Canada's last fully intact ice shelf has suddenly collapsed, forming a Manhattan-sized iceberg",
    "Beijing mobilises invasion craft along coast as Taiwan tensions escalate",
    "The National Park Service warns against sacrificing slower friends in a bear attack",
    "Maine man wins $1M from $25 lottery ticket",
    "Make huge profits without work, earn up to $100,000 a day",
    "i want to fuck you",
    "Check your account balence",
    "you look haat"
]

queryTexts =  [
   "im so horney",
    "im feeling lucky",
    "i like nature and animals",
    "i feel cold",
    "im scared",
    "stocks"
]

# Load a pre-trained Word2Vec model (you can replace this with other embeddings models)
# Make sure you have downloaded and saved the model file locally
# model = gensim.models.Word2Vec.load('path_to_word2vec_model')

def getEmbeddingsFunction(txt):
    return angle.encode({'text': txt}, to_numpy=True)
    return  angle.encode(txt)

def get_embeddings(texts):
    embeddings = []
    for text in texts:
        # Replace this with your actual getEmbeddings function
        embedding = getEmbeddingsFunction(text)
        embeddings.append(embedding)
    return embeddings

# Get embeddings for sourceTexts and queryTexts
source_embeddings = get_embeddings(sourceTexts)
query_embeddings = get_embeddings(queryTexts)

# Perform semantic search for each query
for i, query_text in enumerate(queryTexts):
    print(f"QueryText: {query_text}")
    query_vector = query_embeddings[i]

    # Calculate cosine similarity scores between the query and source texts
    similarity_scores = [cosine_similarity(query_vector, source_vector)[0][0] for source_vector in source_embeddings]

    # Sort the source texts by similarity score and get the top 3 matches
    sorted_matches = sorted(enumerate(similarity_scores), key=lambda x: x[1], reverse=True)[:3]

    # Print the top 3 matches
    for rank, (index, score) in enumerate(sorted_matches, start=1):
        print(f"Match{rank}: {sourceTexts[index]} (Similarity Score: {score:.4f})")
    print()

INFO:AnglE:Prompt is set, the prompt will be automatically applied during the encoding phase. To disable prompt setting, please configure set_prompt(prompt=None)


QueryText: im so horney
Match1: i want to fuck you (Similarity Score: 0.4509)
Match2: you look haat (Similarity Score: 0.4087)
Match3: Check your account balence (Similarity Score: 0.4033)

QueryText: im feeling lucky
Match1: Maine man wins $1M from $25 lottery ticket (Similarity Score: 0.4928)
Match2: Make huge profits without work, earn up to $100,000 a day (Similarity Score: 0.4625)
Match3: i want to fuck you (Similarity Score: 0.4489)

QueryText: i like nature and animals
Match1: The National Park Service warns against sacrificing slower friends in a bear attack (Similarity Score: 0.5142)
Match2: i want to fuck you (Similarity Score: 0.4978)
Match3: Canada's last fully intact ice shelf has suddenly collapsed, forming a Manhattan-sized iceberg (Similarity Score: 0.4017)

QueryText: i feel cold
Match1: Canada's last fully intact ice shelf has suddenly collapsed, forming a Manhattan-sized iceberg (Similarity Score: 0.4358)
Match2: Check your account balence (Similarity Score: 0.4188)
